1、创建表

In [4]:
from pymilvus import  MilvusClient, DataType, MilvusException
import time

# 1. Set up a Milvus client
client = MilvusClient(
    uri="http://222.20.98.71:19530"
)

# 3. Create a collection in customized setup mode
# 3.1. Create schema
schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True,
)
# 3.2. Add fields to schema
schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)  # 主键字段
schema.add_field(field_name="image_embedding", datatype=DataType.FLOAT_VECTOR, dim=96)  # 向量字段
schema.add_field(field_name="col_1", datatype=DataType.INT64)  # 整数字段

# 3.3 Prepare index parameters
index_params = client.prepare_index_params()
# 3.4 Add indexs
index_params.add_index(
    field_name="id",
    index_type="STL_SORT"
)

index_params.add_index(
    field_name="image_embedding", 
    index_type="IVF_FLAT",
    metric_type="L2",
    params={
        "nlist": 1000
    }
)

client.create_collection(
    collection_name="deep_range",
    schema=schema,
    index_params=index_params
)

time.sleep(1)

res = client.get_load_state(
    collection_name="deep_range"
)

print(res)


{'state': <LoadState: Loaded>}


2、插入数据

In [ ]:
import struct
from pymilvus import MilvusClient
import os

# 1. 设置 Milvus 客户端
client = MilvusClient(
    uri="http://222.20.98.71:19530"
)

# 2. 定义集合名称和批量插入的批量大小
collection_name = "deep_range"
batch_size = 1000

def load_and_insert_data(vector_file_path):
    """
    Load vector data from a file and insert it into the Milvus collection.

    Args:
        vector_file_path: Path to the vector file.
    """
    # 打开向量文件
    with open(vector_file_path, "rb") as vector_file:
        batch_data = []
        id_counter = 0  # id 从 1 开始自增

        while True:
            # 读取向量数据
            dim = vector_file.read(4)
            if not dim:
                break  # 文件结束
            dim = struct.unpack('i', dim)[0]
            vector = struct.unpack('f' * dim, vector_file.read(4 * dim))

            # 构造数据条目
            data_entry = {
                "id": id_counter,  # 使用自增的 id
                "image_embedding": list(vector),  # 向量数据
                "col_1": id_counter,  # col_1 的值与 id 相同
            }

            batch_data.append(data_entry)

            # id 自增
            id_counter += 1

            # 每批次插入一次数据
            if len(batch_data) >= batch_size:
                client.insert(collection_name=collection_name, data=batch_data)
                print(f"插入了 {len(batch_data)} 行数据。")
                batch_data = []

        # 插入剩余数据
        if batch_data:
            client.insert(collection_name=collection_name, data=batch_data)
            print(f"插入了 {len(batch_data)} 行数据。")

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), "../../../data/Experiment"))
# 4. 执行数据加载和插入
vector_file_path = os.path.join(ROOT_DIR, "rangefilterData/datasets/deep/deep_base.fvecs")
load_and_insert_data(vector_file_path)

print("数据导入完成。")

# 1分30多秒
# 783MB

3、单线程脚本测试

In [ ]:
from pymilvus import MilvusClient
import time
import numpy as np
import psutil  # 用于监控进程内存
import os

# 获取当前进程
process = psutil.Process(os.getpid())
peak_memory_mb = 0  # 用于记录峰值内存（单位 MB）

# 定义一个函数来更新峰值内存
def update_peak_memory():
    global peak_memory_mb
    memory_info = process.memory_info()
    current_memory_mb = memory_info.rss / 1024 / 1024  # 将字节转换为 MB
    peak_memory_mb = max(peak_memory_mb, current_memory_mb)

# 1. Set up a Milvus client
client = MilvusClient(
    uri="http://222.20.98.71:19530"
)

# 2. Function to read fvecs file
def read_fvecs(file_path, num_vectors):
    data = []
    with open(file_path, 'rb') as f:
        for _ in range(num_vectors):
            dim = np.frombuffer(f.read(4), dtype=np.int32)[0]
            vector = np.frombuffer(f.read(dim * 4), dtype=np.float32)
            data.append(vector.tolist())
    update_peak_memory()  # 检查内存
    return data

def read_txt_file(file_path):
    """读取 .txt 文件并返回查询条件列表"""
    conditions = []
    with open(file_path, "r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            if line:  # 跳过空行
                # 用空格分割，获取两个值作为范围
                parts = line.split()
                if len(parts) == 2:
                    conditions.append((int(parts[0]), int(parts[1])))  # 存储范围值
    return conditions

list_1 = ["2", "8"]
list_3 = [5, 8, 10, 15, 20, 30, 50, 100]
num_vectors = 10000
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), "../../../data/Experiment"))
# 文件路径
fvecs_file = os.path.join(ROOT_DIR, "rangefilterData/datasets/deep/deep_query.fvecs")
data = read_fvecs(fvecs_file, num_vectors)

# File path and number of vectors to load
for i, i_value in enumerate(list_1):
    for j_value in list_3:
        txt_file = os.path.join(ROOT_DIR, f"rangefilterData/query_range/deep/deep-96-euclidean_queries_2pow-{i_value}_ranges.txt")
        conditions = read_txt_file(txt_file)
        print(f"Loaded {len(data)} vectors and {len(conditions)} filter conditions.")

        # Ensure the number of filters matches the number of query vectors
        if len(conditions) != len(data):
            raise ValueError("The number of filters must match the number of query vectors.")

        output_file = os.path.join(os.getcwd(), "result", f"{i_value}_16_200_{j_value}.out")

        # Load the collection
        client.load_collection(collection_name="deep_range")

        # List to store all results
        all_results = []

        # Start timing
        start_time_1 = time.perf_counter()

        # Perform search for each vector
        for k in range(len(conditions)):
            lower_bound, upper_bound = conditions[k]
            conditions_sql = f"col_1 >= {lower_bound} && col_1 <= {upper_bound}"
            start_time = time.perf_counter()
            res = client.search(
                collection_name="deep_range",
                data=[data[k]],
                filter=conditions_sql,
                anns_field="image_embedding",
                limit=10,
                search_params={"metric_type": "L2", "params": {"nprobe": j_value}},
                output_fields=["id"]
            )
            end_time = time.perf_counter()
            print(f"Search completed in {end_time - start_time} seconds.")
            # Extract ids from the result structure
            result_ids = []
            for item in res[0]:
                result_ids.append(str(item["id"] - 1))

            # Join the IDs with space and add to all_results
            all_results.append(" ".join(result_ids))
            update_peak_memory()  # 在每次搜索后检查内存

        # End timing   
        end_time_1 = time.perf_counter()
        # Write all results to the output file
        with open(output_file, 'w') as f_out:
            # 写入搜索结果
            for result in all_results:
                f_out.write(result + "\n")
            # 追加 QPS 和 Peak RES memory
            f_out.write(f"QPS: {10000 / (end_time_1 - start_time_1):.2f}\n")
            f_out.write(f"Peak RES memory usage: {peak_memory_mb:.2f} MB\n")

4、单线程脚本计算召回率和QPS

In [ ]:
import numpy as np

def read_ivecs(fname):
    with open(fname, "rb") as f:
        data = []
        while True:
            try:
                # Read the dimension
                width = np.fromfile(f, 'int32', 1)[0]
                
                # Read the vector data
                vector = np.fromfile(f, 'int32', width)
                
                # Keep all elements as they are
                data.append(vector)
            except IndexError:
                break  # End of file

    return np.array(data, dtype=object)  # 使用dtype=object因为每行长度可能不同
def read_txt(fname):
    """读取以空格分隔的文本文件，每行包含10个元素"""
    with open(fname, "r") as f:
        data = []
        for line in f:
            # 将每行的数字解析为整数列表
            vector = list(map(int, line.strip().split()))
            data.append(vector[:10])  # 每行只取前10个元素
    return np.array(data)


def get_effective_size(vector):
    """计算向量中有效数字的个数（遇到-1后视为无效）"""
    for i, val in enumerate(vector):
        if val == -1:
            return i  # 返回第一个-1出现的位置作为有效长度
    return len(vector)  # 如果没有-1，返回完整长度

def read_output_file(fname):
    with open(fname, 'r') as f:
        lines = f.readlines()
        if len(lines) > 1:
            lines = lines[:-1]
        return [list(map(int, line.split())) for line in lines]

def extract_qps(fname):
    with open(fname, 'r') as f:
        lines = f.readlines()
        if lines:
            last_line = lines[-1].strip()
            if "QPS:" in last_line:
                qps_part = last_line.split("QPS:")[1].split("Peak")[0].strip()
                return qps_part
    return "N/A"

def extract_res(fname):
    with open(fname, 'r') as f:
        lines = f.readlines()
        if lines:
            last_line = lines[-1].strip()
            if "Peak RES memory usage:" in last_line:
                res_part = last_line.split("Peak RES memory usage:")[1].split("MB")[0].strip()
                return res_part
    return "N/A"

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), "../../../data/Experiment"))
recall_file = os.path.join(os.getcwd(), "result", f"single_qps.out")

# 打开文件用于写入结果
with open(recall_file, "a") as output_file:
    list_1 = [2, 8]
    list_2 = [5, 8, 10, 15, 20, 30, 50, 100]
    
    for i_value in list_1:
        for j_value in list_2:
            # Read ground truth
            gt_file = os.path.join(ROOT_DIR, f"rangefilterData/gt/deep/gt-query_set_{i_value}.ivecs")
            gt_data = read_ivecs(gt_file)
            # print(gt_data)
            result_file = os.path.join(os.getcwd(), "result", f"{i_value}_16_200_{j_value}.out")
            result_data = read_output_file(result_file)

            # Extract QPS value
            qps_value = extract_qps(result_file)
            res = extract_res(result_file)

            # Calculate recall
            total_queries = len(gt_data)
            correct_matches = 0
            total_possible_matches = 0

            # For each query in ground truth
            for i, gt_row in enumerate(gt_data):
                gt_effective_size = get_effective_size(gt_row)  # 获取有效长度
                print(gt_effective_size)
                # 计算当前查询的正确匹配数，只考虑有效部分
                correct_matches += sum(1 for gt_val in gt_row[:gt_effective_size] 
                                     if gt_val - 1 in result_data[i][:gt_effective_size])
                total_possible_matches += gt_effective_size  # 累计所有可能的匹配总数

            # Recall calculation using effective number of elements
            recall = correct_matches / total_possible_matches if total_possible_matches > 0 else 0

            # 写入结果到文件
            output_file.write(f"Recall Rate {i_value:<6} and {j_value:<4}: {recall:>6.4f}, QPS: {float(qps_value):>8.2f}, RES: {float(res):>8.2f} MB\n")